In [14]:
import numpy as np
import matplotlib.pyplot as plt
import csv

In [15]:
def compute_iou(pred, gt):
    """
    Compute Intersection over Union (IoU)
    :param pred: numpy array of predictions
    :param gt: numpy array of ground truth
    :return: IoU score
    """
    intersection = np.logical_and(pred == 1, gt == 1)
    union = np.logical_or(pred == 1, gt == 1)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def compute_precision(pred, gt):
    """
    Compute Precision
    :param pred: numpy array of predictions
    :param gt: numpy array of ground truth
    :return: Precision score
    """
    true_positive = np.sum((pred == 1) & (gt == 1))
    false_positive = np.sum((pred == 1) & (gt != 1))
    precision = true_positive / (true_positive + false_positive)
    return precision

def compute_recall(pred, gt):
    """
    Compute Recall
    :param pred: numpy array of predictions
    :param gt: numpy array of ground truth
    :return: Recall score
    """
    true_positive = np.sum((pred == 1) & (gt == 1))
    false_negative = np.sum((pred != 1) & (gt == 1))
    recall = true_positive / (true_positive + false_negative)
    return recall

In [25]:
def get_evaluation_metrics(model_predictions, ground_truths):
    iou = np.round(compute_iou(pred=model_predictions, gt=ground_truths), 5)
    precision = np.round(compute_precision(pred=model_predictions, gt=ground_truths), 5)
    recall = np.round(compute_recall(pred=model_predictions, gt=ground_truths), 5)

    true_positive = np.sum((model_predictions == 1) & (ground_truths == 1))
    false_positive = np.sum((model_predictions == 1) & (ground_truths != 1))    
    true_negative = np.sum((model_predictions != 1) & (ground_truths != 1))
    false_negative = np.sum((model_predictions != 1) & (ground_truths == 1))

    true_positive_ratio = np.round(true_positive/(true_positive+false_positive+true_negative+false_negative), 4)
    false_positive_ratio = np.round(false_positive/(true_positive+false_positive+true_negative+false_negative), 4)
    true_negative_ratio = np.round(true_negative/(true_positive+false_positive+true_negative+false_negative), 4)
    false_negative_ratio = np.round(false_negative/(true_positive+false_positive+true_negative+false_negative), 4)

    return iou, precision, recall, true_positive, false_positive, true_negative, false_negative, true_positive_ratio, false_positive_ratio, true_negative_ratio, false_negative_ratio

$\text{IoU} = \frac{|A \cap B|}{|A \cup B|}$

$\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$

$\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$


In [29]:
def evaluate_and_save_results(models, experiment_date):
    
    ground_truths = np.load('/home/tryptag/kiran/CellSegmentation/unet_v20_Testing/test_masks_TP.npy')

    csv_file = f"/datapot/tryptag/kiran/Predictions_using_models/{experiment_date}.csv"

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Model', 'IoU', 'Precision', 'Recall', 'True Positive', 'False Positive', 'True Negative', 'False Negative', 'True Positive Ratio', 'False Positive Ratio', 'True Negative Ratio', 'False Negative Ratio'])

        # Iterating through the models
        for model in models:
            model_predictions = np.load(f"/datapot/tryptag/kiran/Predictions_using_models/{model}_predictions_{experiment_date}.npy")
            iou, precision, recall, tp, fp, tn, fn, tpr, fpr, tnr, fnr  = get_evaluation_metrics(model_predictions, ground_truths)
            writer.writerow([model, iou, precision, recall, tp, fp, tn, fn, tpr, fpr, tnr, fnr])

In [31]:
# experiment_date = '20June2024'
models = ['ilastik', 'tryptag', 'combined']

# experiment_dates = ['30May2024', '14June2024', '20June2024']
experiment_dates = ['28June2024_GDL', '28June2024_LogCoshGDL']
for experiment_date in experiment_dates:
    evaluate_and_save_results(models=models, experiment_date=experiment_date)